测试 albert_small 情感分类  
https://github.com/bojone/bert4keras/blob/master/examples/task_sentiment_albert.py

# part1:配置环境 + 加载文件

In [3]:
#! pip install keras_bert
! pip install git+https://www.github.com/bojone/bert4keras.git
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.__version__)
tf.test.gpu_device_name()
device_lib.list_local_devices()
!nvidia-smi

  Cloning https://www.github.com/bojone/bert4keras.git to /tmp/pip-req-build-m9n9lr6n
  Running command git clone -q https://www.github.com/bojone/bert4keras.git /tmp/pip-req-build-m9n9lr6n
  Created wheel for bert4keras: filename=bert4keras-0.8.3-cp36-none-any.whl size=41795 sha256=652c7ec7d8410062cb35deb6f6a3846f10759f74a443b444ba295e88add8391e
  Stored in directory: /tmp/pip-ephem-wheel-cache-t3l54x1y/wheels/12/58/83/8ff5c864b80c860e6d9e9e0d90c04fafca05d01d21f9f6fcba
Successfully built bert4keras
2.2.0
Wed Jul 22 01:17:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      | 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### 加载数据文件

In [5]:
! git clone https://github.com/bojone/bert4keras.git
! unzip -d '/content/sample_data' "/content/bert4keras/examples/datasets/sentiment.zip"

Cloning into 'bert4keras'...
remote: Enumerating objects: 5684, done.
remote: Total 5684 (delta 0), reused 0 (delta 0), pack-reused 5684
Receiving objects: 100% (5684/5684), 9.20 MiB | 23.38 MiB/s, done.
Resolving deltas: 100% (3741/3741), done.
Archive:  /content/bert4keras/examples/datasets/sentiment.zip
   creating: /content/sample_data/sentiment/
  inflating: /content/sample_data/sentiment/sentiment.test.data  
  inflating: /content/sample_data/sentiment/sentiment.train.data  
  inflating: /content/sample_data/sentiment/sentiment.valid.data  


#### 1.1 加载ALbert_small

In [6]:
! wget https://storage.googleapis.com/albert_zh/albert_small_zh_google.zip
! unzip -o "/content/albert_small_zh_google.zip" -d '/content/albert_small_zh_google'

--2020-07-22 01:52:33--  https://storage.googleapis.com/albert_zh/albert_small_zh_google.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.124.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.124.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53458815 (51M) [application/zip]
Saving to: ‘albert_small_zh_google.zip’

albert_small_zh_goo 100%[===================>]  50.98M   103MB/s    in 0.5s    

2020-07-22 01:52:34 (103 MB/s) - ‘albert_small_zh_google.zip’ saved [53458815/53458815]

Archive:  /content/albert_small_zh_google.zip
  inflating: /content/albert_small_zh_google/albert_config_small_google.json  
  inflating: /content/albert_small_zh_google/albert_model.ckpt.data-00000-of-00001  
  inflating: /content/albert_small_zh_google/albert_model.ckpt.index  
  inflating: /content/albert_small_zh_google/albert_model.ckpt.meta  
  inflating: /content/albert_small_zh_google/c

#### 1.2 Albert_large_zh

该版本不能被直接使用需要转换  
https://github.com/bojone/albert_zh

In [ ]:
## 百度下载的 albert_large
! unzip -o "/content/drive/My Drive/local_copy/albert_large_google_zh.zip" -d '/content/albert_large_google_zh'

Archive:  /content/drive/My Drive/local_copy/albert_large_google_zh.zip
   creating: /content/albert_large_google_zh/albert_large_google_zh/
  inflating: /content/albert_large_google_zh/albert_large_google_zh/albert_model.ckpt.data-00000-of-00001  
  inflating: /content/albert_large_google_zh/albert_large_google_zh/albert_model.ckpt.index  
  inflating: /content/albert_large_google_zh/albert_large_google_zh/checkpoint  
  inflating: /content/albert_large_google_zh/albert_large_google_zh/vocab.txt  
  inflating: /content/albert_large_google_zh/albert_large_google_zh/albert_config.json  


#### 1.3 Chinese-BERT-wwm


In [ ]:
# 加载drive中的bert checkpoint
#! wget https://github.com/ymcui/Chinese-BERT-wwm
! unzip "/content/drive/My Drive/Colab Notebooks/chinese_wwm_L-12_H-768_A-12.zip" 

Archive:  /content/drive/My Drive/Colab Notebooks/chinese_wwm_L-12_H-768_A-12.zip
   creating: publish/
  inflating: publish/vocab.txt       
  inflating: publish/bert_model.ckpt.index  
  inflating: publish/bert_model.ckpt.data-00000-of-00001  
  inflating: publish/bert_config.json  
  inflating: publish/bert_model.ckpt.meta  


# part2 配置全局参数

In [7]:
import json
from tqdm import tqdm
import os, re
import numpy as np
import pandas as pd
import codecs

import numpy as np
import keras
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense
from bert4keras.optimizers import *

set_gelu('tanh')  # 切换gelu版本

num_classes = 2
maxlen = 128
batch_size = 64
# 1 训练数据文件路径
data_dir = "/content/sample_data/sentiment"
train_data_dir = data_dir + "/sentiment.train.data"
valid_data_dir = data_dir + "/sentiment.valid.data"
test_data_dir = data_dir + "/sentiment.test.data"

print(os.path.exists(train_data_dir))

# 3 训练模型文件路径
checkpoint_dir = 'checkpoint/subject_extract2'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
# 4 结果输出路径    
outputdir = '/content/drive/My Drive'

True


### 2.1 albert_small_google

In [8]:

config_path = '/content/albert_small_zh_google/albert_config_small_google.json'
checkpoint_path = '/content/albert_small_zh_google/albert_model.ckpt'
dict_path = '/content/albert_small_zh_google/vocab.txt'


### 2.2 albert_large_google

In [ ]:
model_dir = "/content/albert_large_google_zh/albert_large_google_zh"
config_path = model_dir + '/albert_config.json'
checkpoint_path = model_dir + '/albert_model.ckpt'
dict_path = model_dir + '/vocab.txt'

### 2.3 bert_base

In [ ]:
config_path = './publish/bert_config.json'
checkpoint_path = './publish/bert_model.ckpt'
dict_path = './publish/vocab.txt'
 
print('load checkpoint path : ',os.path.exists(config_path))

load checkpoint path :  True


# part 3 模型部分

## 数据构建

In [9]:

def load_data(filename):
    D = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            text, label = l.strip().split('\t')
            D.append((text, int(label)))
    return D


# 加载数据集
train_data = load_data(train_data_dir)
valid_data = load_data(valid_data_dir)
test_data = load_data(test_data_dir)

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)
test_generator = data_generator(test_data, batch_size)


In [10]:
# 查看数据
tag = 0
for i in train_generator.forfit():
  tag +=1
  if tag>1:
    break
  #print(i,len(i))
  #print(i[0],'\n',i[1])
  #print(i[0][0],'\n',i[0][0].shape)
  #print(i[0][1],'\n',i[0][1].shape)
  #print(i[1],'\n',i[1].shape)
  print('ids : {} \ntext:{}\nlabel:{}'.format(i[0][0][0],tokenizer.decode(i[0][0][0]),i[1][0]))

ids : [ 101 3238 8024 5632 2346 1405 5632 2346  749 8024 1157 1157 3018 3209
 4635 8024 6821 1372 3322 2094 3064 8388 3198 7509 7030  679 5543 4500
 5011 6444 4638 8024 6206 4500 2797 3322 2340  904 4638  677  678 7241
 6444 5688  511 2597  679 2533 2769 4500 5011 4157  749 3187 3144 3613
 3064 3123 1690 3297 1400 7481 3300  702 1898 7509 1920 2207 1377 6444
 5688 3403 2562 6963 3766 1353 2418 8024  809  711 8388 1776 2957  749
 8013 1963 3362  679 5543 4500  117  711  784  720 6820 6206 6821 3416
 2094  976 4518 7481  117 4696 3221 4638  106 5314 3322 1351  812 2990
  702 7008 8024 1920 2157 3800 2692  671  678 1416  102    0    0    0
    0    0] 
text:晕，自己吓自己了，刚刚搞明白，这只机子播mp3时音量不能用笔调的，要用手机左侧的上下键调节。怪不得我用笔点了无数次播放器最后面有个声音大小可调节标志都没反应，以为mp3坏掉了！如果不能用, 为什么还要这样子做界面, 真是的! 给机友们提个醒，大家注意一下吧
label:[0]


##  模型构建

### ALBERT

#### 不训练层

In [ ]:
K.clear_session()
# 加载预训练模型
## return_keras_model=False 不训练层
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='albert',
    return_keras_model=False,
)

output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)#只取cls值
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。


#### 训练层--代码没改

In [ ]:
K.clear_session()
# 加载预训练模型
## return_keras_model=False 不训练层
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='albert',
    return_keras_model=False,
)

output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)#只取cls值
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。


### Bert

In [ ]:
K.clear_session()
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    return_keras_model=True,
)
layer_name = 'Transformer-3-FeedForward-Norm'                                    


output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.get_layer(layer_name).output)#只取cls值
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02)
)(output)

model = keras.models.Model(bert.input, output)
model.summary()

### callback

In [12]:

def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total


class Evaluator(keras.callbacks.Callback):
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
        test_acc = evaluate(test_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )



### Optimizer  
    Adam AdaFactor AdaX   

优秀的老手用SGD精调

### 优化器策略  
#### 1 权重衰减
    extend_with_weight_decay  

#### 2 层自适应学习率
    extend_with_layer_adaptation

#### 3 分段线性学习率
    extend_with_piecewise_linear_lr  

#### 4 梯度累积
    extend_with_gradient_accumulation  

#### 5 带有look ahead的优化器   
    extend_with_lookahead  
https://arxiv.org/abs/1907.08610
steps_per_slow_update: 即论文中的k；
slow_step_size: 即论文中的alpha。

#### 6 懒惰更新 
    extend_with_lazy_optimization   
使得部分权重（尤其是embedding）只有在梯度不等于0时
        才发生更新。  

#### 7 EMA（权重滑动平均）
    extend_with_exponential_moving_average  


#### 3.1 Adam

In [13]:
model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=Adam(learning_rate=1e-4),
    metrics=['accuracy'],
)
evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=10,
    callbacks=[evaluator]
)

model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
264/264 [==============================] - 151s 572ms/step - loss: 0.3020 - accuracy: 0.8769
val_acc: 0.91994, best_val_acc: 0.91994, test_acc: 0.91852

Epoch 2/10
264/264 [==============================] - 147s 556ms/step - loss: 0.1612 - accuracy: 0.9421
val_acc: 0.93984, best_val_acc: 0.93984, test_acc: 0.93321

Epoch 3/10
264/264 [==============================] - 147s 556ms/step - loss: 0.0949 - accuracy: 0.9681
val_acc: 0.93226, best_val_acc: 0.93984, test_acc: 0.93415

Epoch 4/10
264/264 [==============================] - 147s 555ms/step - loss: 0.0594 - accuracy: 0.9818
val_acc: 0.92184, best_val_acc: 0.93984, test_acc: 0.92942

Epoch 5/10
264/264 [==============================] - 147s 556ms/step - loss: 0.0459 - accuracy: 0.9849
val_acc: 0.93652, best_val_acc: 0.93984, test_acc: 0.94173

Epoch 6/10
264/264 [==============================] - 147s 556ms/step - loss: 0.0389 - accuracy: 0.9861
val_acc: 0.94079, best_val_acc: 0.94079, test_acc: 0.94315

Epoch 7/10
264/2

##### 3.1.1 线性AdamLR
可以分段调控 ，线性变化

In [ ]:
AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
        1000: 1,
        2000: 0.1
    }),
    metrics=['accuracy'],
)

evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)

model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
264/264 [==============================] - 80s 302ms/step - loss: 0.4689 - accuracy: 0.7638
val_acc: 0.88726, best_val_acc: 0.88726, test_acc: 0.88205

Epoch 2/5
264/264 [==============================] - 80s 304ms/step - loss: 0.2379 - accuracy: 0.9106
val_acc: 0.92468, best_val_acc: 0.92468, test_acc: 0.91757

Epoch 3/5
264/264 [==============================] - 81s 306ms/step - loss: 0.1656 - accuracy: 0.9418
val_acc: 0.93368, best_val_acc: 0.93368, test_acc: 0.92752

Epoch 4/5
264/264 [==============================] - 81s 306ms/step - loss: 0.1188 - accuracy: 0.9582
val_acc: 0.93321, best_val_acc: 0.93368, test_acc: 0.92942

Epoch 5/5
264/264 [==============================] - 81s 307ms/step - loss: 0.0707 - accuracy: 0.9765
val_acc: 0.94173, best_val_acc: 0.94173, test_acc: 0.93321

final test acc: 0.933207



##### 3.1.2 EMA

In [ ]:

AdamEMA = extend_with_exponential_moving_average(Adam, name='AdamEMA')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=AdamEMA(learning_rate=1e-4),
    metrics=['accuracy'],
)


In [ ]:

evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)

model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
264/264 [==============================] - 31s 116ms/step - loss: 0.7227 - accuracy: 0.5087
val_acc: 0.49124, best_val_acc: 0.49124, test_acc: 0.49834

Epoch 2/5
264/264 [==============================] - 28s 108ms/step - loss: 0.7231 - accuracy: 0.5087
val_acc: 0.49124, best_val_acc: 0.49124, test_acc: 0.49834

Epoch 3/5
264/264 [==============================] - 28s 107ms/step - loss: 0.7228 - accuracy: 0.5087
val_acc: 0.49124, best_val_acc: 0.49124, test_acc: 0.49834

Epoch 4/5
264/264 [==============================] - 28s 108ms/step - loss: 0.7230 - accuracy: 0.5087
val_acc: 0.49124, best_val_acc: 0.49124, test_acc: 0.49834

Epoch 5/5
264/264 [==============================] - 28s 108ms/step - loss: 0.7231 - accuracy: 0.5087
val_acc: 0.49124, best_val_acc: 0.49124, test_acc: 0.49834

final test acc: 0.498342



#####3.1.3 梯度累计 

In [ ]:

AdamGACC = extend_with_gradient_accumulation(Adam, name='AdamGACC')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=AdamGACC(learning_rate=1e-4),
    metrics=['accuracy'],
)


In [ ]:
evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)

#model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
264/264 [==============================] - 89s 337ms/step - loss: 0.4022 - accuracy: 0.8081
val_acc: 0.90242, best_val_acc: 0.90242, test_acc: 0.90289

Epoch 2/5
264/264 [==============================] - 85s 323ms/step - loss: 0.1898 - accuracy: 0.9279
val_acc: 0.92942, best_val_acc: 0.92942, test_acc: 0.92184

Epoch 3/5
264/264 [==============================] - 85s 323ms/step - loss: 0.0912 - accuracy: 0.9666
val_acc: 0.92326, best_val_acc: 0.92942, test_acc: 0.92468

Epoch 4/5
264/264 [==============================] - 85s 323ms/step - loss: 0.0388 - accuracy: 0.9867
val_acc: 0.94458, best_val_acc: 0.94458, test_acc: 0.93273

Epoch 5/5
264/264 [==============================] - 85s 322ms/step - loss: 0.0263 - accuracy: 0.9910
val_acc: 0.93084, best_val_acc: 0.94458, test_acc: 0.92136

final test acc: 0.921364



##### 3.3.4 层自适应
extend_with_layer_adaptation

In [ ]:
AdamLA = extend_with_layer_adaptation(Adam, name='AdamLA')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=AdamLA(learning_rate=1e-4),
    metrics=['accuracy'],
)
evaluator = Evaluator()
model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)
model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
264/264 [==============================] - 89s 338ms/step - loss: 0.3888 - accuracy: 0.8168
val_acc: 0.91805, best_val_acc: 0.91805, test_acc: 0.91047

Epoch 2/5
264/264 [==============================] - 86s 325ms/step - loss: 0.1841 - accuracy: 0.9318
val_acc: 0.93605, best_val_acc: 0.93605, test_acc: 0.93368

Epoch 3/5
264/264 [==============================] - 86s 325ms/step - loss: 0.1074 - accuracy: 0.9622
val_acc: 0.94458, best_val_acc: 0.94458, test_acc: 0.93463

Epoch 4/5
264/264 [==============================] - 86s 325ms/step - loss: 0.0598 - accuracy: 0.9808
val_acc: 0.94126, best_val_acc: 0.94458, test_acc: 0.93510

Epoch 5/5
264/264 [==============================] - 86s 324ms/step - loss: 0.0335 - accuracy: 0.9892
val_acc: 0.93794, best_val_acc: 0.94458, test_acc: 0.93652

final test acc: 0.934628



##### 3.3.5 权重衰减  
extend_with_weight_decay

In [ ]:
AdamWdecay = extend_with_weight_decay(Adam, name='AdamWdecay')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=AdamWdecay(learning_rate=1e-4, ),
    metrics=['accuracy'],
)
evaluator = Evaluator()
model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)
model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
264/264 [==============================] - 86s 326ms/step - loss: 0.1451 - accuracy: 0.9462
val_acc: 0.94315, best_val_acc: 0.94315, test_acc: 0.93558

Epoch 2/5
264/264 [==============================] - 83s 315ms/step - loss: 0.0742 - accuracy: 0.9739
val_acc: 0.93842, best_val_acc: 0.94315, test_acc: 0.93321

Epoch 3/5
264/264 [==============================] - 83s 316ms/step - loss: 0.0477 - accuracy: 0.9834
val_acc: 0.93368, best_val_acc: 0.94315, test_acc: 0.94031

Epoch 4/5
264/264 [==============================] - 83s 316ms/step - loss: 0.0336 - accuracy: 0.9884
val_acc: 0.94268, best_val_acc: 0.94315, test_acc: 0.93937

Epoch 5/5
264/264 [==============================] - 83s 315ms/step - loss: 0.0265 - accuracy: 0.9898
val_acc: 0.93605, best_val_acc: 0.94315, test_acc: 0.93842

final test acc: 0.935576



#### 3.2 SGD


In [15]:
from keras.optimizers import SGD

In [16]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=SGD(learning_rate=1e-3, decay=1e-6, momentum=0.9),
    metrics=['accuracy'],
)
evaluator = Evaluator()
model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)
model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
264/264 [==============================] - 148s 560ms/step - loss: 0.0135 - accuracy: 0.9955
val_acc: 0.93700, best_val_acc: 0.93700, test_acc: 0.94505

Epoch 2/5
264/264 [==============================] - 146s 552ms/step - loss: 0.0044 - accuracy: 0.9986
val_acc: 0.94268, best_val_acc: 0.94268, test_acc: 0.94268

Epoch 3/5
264/264 [==============================] - 146s 552ms/step - loss: 0.0021 - accuracy: 0.9992
val_acc: 0.94363, best_val_acc: 0.94363, test_acc: 0.94410

Epoch 4/5
264/264 [==============================] - 145s 551ms/step - loss: 0.0016 - accuracy: 0.9991
val_acc: 0.94315, best_val_acc: 0.94363, test_acc: 0.94505

Epoch 5/5
264/264 [==============================] - 145s 551ms/step - loss: 0.0014 - accuracy: 0.9992
val_acc: 0.94315, best_val_acc: 0.94363, test_acc: 0.94458

final test acc: 0.944102



In [ ]:
 

SGDLhead = extend_with_weight_decay(SGD, name='SGDLhead')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=SGDLhead(learning_rate=1e-3, decay=1e-6, momentum=0.9),
    metrics=['accuracy'],
)
evaluator = Evaluator()
model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)
model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

#### 3.3 AdaFactor
这个优化器lr batch_size 需要大一点

In [ ]:
!git clone https://github.com/bojone/adafactor.git

Cloning into 'adafactor'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [ ]:
from adafactor.adafactor import AdaFactor

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=AdaFactor(1e-3),
    metrics=['accuracy'],
)

evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)

model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 11/264 [>.............................] - ETA: 2:20 - loss: nan - accuracy: 0.4673

KeyboardInterrupt: ignored

In [ ]:
AdaFLR = extend_with_piecewise_linear_lr(AdaFactor, name='AdaFLR')
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=AdaFLR(learning_rate=1e-2, lr_schedule={
        1000: 1,
        3000: 0.1
    }),
    metrics=['accuracy'],
)

evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)

#model.load_weights('best_model.weights')


print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 79/264 [=======>......................] - ETA: 1:07 - loss: nan - accuracy: 0.4909

KeyboardInterrupt: ignored

#### 3.4 Adax

In [ ]:
!git clone https://github.com/bojone/adax.git

Cloning into 'adax'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [ ]:
from adax.adax import AdaX

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-3),  # 用足够小的学习率
    optimizer=AdaX(1e-4),
    metrics=['accuracy'],
)

evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)

#model.load_weights('best_model.weights')
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
264/264 [==============================] - 151s 572ms/step - loss: 0.5147 - accuracy: 0.7696
val_acc: 0.77546, best_val_acc: 0.77546, test_acc: 0.76883

Epoch 2/5
264/264 [==============================] - 157s 593ms/step - loss: 0.6946 - accuracy: 0.5071
val_acc: 0.50876, best_val_acc: 0.77546, test_acc: 0.50166

Epoch 3/5
227/264 [========================>.....] - ETA: 21s - loss: 0.6946 - accuracy: 0.5162

##### 3.4.1 线性分段

In [ ]:

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=AdaXLR(learning_rate=1e-3, lr_schedule={
        1000: 1,
        2000: 0.1
    }),
    metrics=['accuracy'],
)

evaluator = Evaluator()

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=5,
    callbacks=[evaluator]
)

#model.load_weights('best_model.weights')
AdaXLR = extend_with_piecewise_linear_lr(Adam, name='AdaXLR')

print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
264/264 [==============================] - 86s 327ms/step - loss: 0.0569 - accuracy: 0.9803
val_acc: 0.87210, best_val_acc: 0.87210, test_acc: 0.87636

Epoch 2/5
264/264 [==============================] - 84s 317ms/step - loss: 0.1605 - accuracy: 0.9404
val_acc: 0.90715, best_val_acc: 0.90715, test_acc: 0.90810

Epoch 3/5
264/264 [==============================] - 84s 317ms/step - loss: 0.2073 - accuracy: 0.9225
val_acc: 0.88868, best_val_acc: 0.90715, test_acc: 0.87541

Epoch 4/5
264/264 [==============================] - 84s 317ms/step - loss: 0.3851 - accuracy: 0.8389
val_acc: 0.84462, best_val_acc: 0.90715, test_acc: 0.83562

Epoch 5/5
264/264 [==============================] - 84s 317ms/step - loss: 0.3218 - accuracy: 0.8721
val_acc: 0.85836, best_val_acc: 0.90715, test_acc: 0.85505

final test acc: 0.855045



# part 4 错误分析

### train

In [ ]:
# 打印错误数
num = 6
tag = 0
for x_true, y_true in test_generator:
    tag +=1
    if tag>num:
      break
    #print(x_true[0].shape)
    y_pred = model.predict(x_true).argmax(axis=1)
    y_true = y_true[:, 0]
    #print(y_true)
    #print(y_pred)
    #error_pred = x_true[0][y_true!=y_pred]
    error_ids = np.array(list(range(len(y_true))))[y_true!=y_pred]

    print(error_ids)
    for ids in error_ids:
      print('text :{}\npredict:{}, ture:{}\n'.format(tokenizer.decode(x_true[0][ids]),y_pred[ids],y_true[ids]))

# 重启

In [ ]:
# 重启
!apt install psmisc
!sudo fuser /dev/nvidia*


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  psmisc
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 52.5 kB of archives.
After this operation, 266 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 psmisc amd64 23.1-1ubuntu0.1 [52.5 kB]
Fetched 52.5 kB in 0s (1,074 kB/s)
Selecting previously unselected package psmisc.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../psmisc_23.1-1ubuntu0.1_amd64.deb ...
Unpacking psmisc (23.1-1ubuntu0.1) ...
Setting up psmisc (23.1-1ubuntu0.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
/dev/nvidia0:          122m
/dev/nvidiactl:        122m
/dev/nvidia-uvm:       122m


In [ ]:
! kill -9 122